In [60]:
import pandas as pd
import jieba
testjq = pd.read_excel('D:\泰迪杯\c题\C题测试数据\景区评论（测试数据）.xlsx')
jqscore = pd.read_excel('D:/泰迪杯/c题/附件2/景区评分.xlsx')
testjq

,景区名称,评论日期,评论内容
0,A51,2020-04-05,整体还可以，适合家人朋友一起出游，门票有点小贵，里面各项游乐设施均需要另外收费，这个季节花开...
1,A51,2020-01-04,**生态园动物园这个地方挺好的，取票方便，服务态度又好，里面风景好，动物也好看，适合居家旅行...
2,A51,2020-12-13,环境一个字差，服务态度差，特别是钓鱼的地方，当你钓鱼后向工作人员要一个塑料袋只会给你一个，当...
3,A51,2020-10-06,你好，亲，有护士执业有免费吗？
4,A51,2020-05-07,景区里面还是太小了，没有什么好看和玩的。门票还要那么贵！
...,...,...,...
1934,A60,2015-12-31,无数人的名字都是和**的名字或者是字有关，可以看出这里的人们是多么爱戴这位大伟人。
1935,A60,2015-04-16,潮州很多路名 街名都与**有关**便是如此引文**当年来潮州做官 治理得很好 所以市民为了纪...
1936,A60,2015-12-17,从广济桥这边岸一直走过去湘江对岸，就可以直接上去韩公祠，里面其实很大的，需要慢慢逛。
1937,A60,2015-10-04,好，有意义！交通也方便！


In [61]:
testjq.index=testjq['景区名称']
x = [f'A{x}{y}' for x in range(5,7) for y in range(0,10)][1:11]
df = [testjq.loc[f'{x}']['评论内容'] for x in x]

In [62]:
df=[str(i) for i in df]

In [63]:
import re
#加载停用词
stoplist= [word.strip() for word in open('stopwords.txt',encoding='gb2312').readlines()]
# print(stoplist)
sentences_list = df
# 对句子进行分词
def seg_depart(sentence):
    # 去掉非汉字字符
    sentence = re.sub(r'[^\u4e00-\u9fa5]+','',sentence)
    sentence_depart = jieba.cut(sentence.strip())
    word_list = []
    for word in sentence_depart:
        if word not in stoplist:
            word_list.append(word)
    # 如果句子整个被过滤掉了，如：'02-2717:56'被过滤，那就返回[],保持句子的数量不变
    return word_list

sentence_word_list = []
for sentence in sentences_list:
    line_seg = seg_depart(sentence)
    sentence_word_list.append(line_seg)

df = sentences_list

In [64]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
 
#该类会统计每个词语的tf-idf权值
tf_idf_transformer = TfidfVectorizer()

df = tf_idf_transformer.fit_transform(df)
type(df)

scipy.sparse.csr.csr_matrix

In [65]:
df

<10x420 sparse matrix of type '<class 'numpy.float64'>'
	with 478 stored elements in Compressed Sparse Row format>

In [296]:
from sklearn.ensemble import BaggingRegressor
model_BaggingRegressor = BaggingRegressor(n_estimators=27,max_samples=1.0,max_features=1.0,n_jobs=None)
joblib.dump(model_BaggingRegressor,'B_model')

['B_model']

In [256]:
from sklearn.externals import joblib
y_test = jqscore.sample(n=10,axis=0)

In [257]:
model = joblib.load('B_model')#加载模型

In [260]:
y_test1 = y_test['总得分']
model.fit(df,y_test1)
y_hat1 = model.predict(df)
print("总得分得分评分：",y_hat1)

总得分得分评分： [4.55148148 4.71314815 4.59       4.5762963  4.44314815 4.44944444
 4.53018519 4.31666667 4.72722222 4.56277778]


In [261]:
y_test2 = y_test['服务得分']
model.fit(df,y_test2)
y_hat2 = model.predict(df)
print("服务得分：",y_hat2)

服务得分： [3.95185185 4.51111111 3.9037037  3.89259259 4.02222222 3.84074074
 4.16296296 3.9        4.41111111 3.8962963 ]


In [262]:
y_test3 = y_test['位置得分']
model.fit(df,y_test3)
y_hat3 = model.predict(df)
print("位置得分：",y_hat3)

位置得分： [4.9        4.9        4.9        4.9        4.9        4.9
 4.75185185 4.51111111 4.9        4.9       ]


In [263]:
y_test4 = y_test['设施得分']
model.fit(df,y_test4)
y_hat4 = model.predict(df)
print("设施得分：",y_hat4)

设施得分： [4.7        4.6        4.86666667 4.86666667 4.67407407 4.4
 4.35555556 4.21111111 4.72222222 4.83333333]


In [264]:
y_test5 = y_test['卫生得分']
model.fit(df,y_test5)
y_hat5 = model.predict(df)
print("卫生得分：",y_hat5)

卫生得分： [4.9        4.9        4.9        4.9        4.43333333 4.9
 4.9        4.66296296 4.9        4.9       ]


In [265]:
y_test6 = y_test['性价比得分']
model.fit(df,y_test6)
y_hat6 = model.predict(df)
print("性价比得分：",y_hat6)

性价比得分： [4.61111111 4.84444444 4.78518519 4.72222222 4.77407407 4.61481481
 4.71111111 4.61851852 4.7962963  4.63333333]


In [288]:
import pandas as pd
dataset = pd.DataFrame({'景区名称':x,'总得分': y_hat1, '服务得分': y_hat2, '位置得分': y_hat3, '设施得分': y_hat4, '卫生得分': y_hat5, '性价比得分': y_hat6,})

In [294]:
dataset.set_index(['景区名称'],inplace=True)
dataset

,总得分,服务得分,位置得分,设施得分,卫生得分,性价比得分
景区名称,,,,,,
A51,4.551481,3.951852,4.900000,4.700000,4.900000,4.611111
A52,4.713148,4.511111,4.900000,4.600000,4.900000,4.844444
A53,4.590000,3.903704,4.900000,4.866667,4.900000,4.785185
A54,4.576296,3.892593,4.900000,4.866667,4.900000,4.722222
A55,4.443148,4.022222,4.900000,4.674074,4.433333,4.774074
A56,4.449444,3.840741,4.900000,4.400000,4.900000,4.614815
A57,4.530185,4.162963,4.751852,4.355556,4.900000,4.711111
A58,4.316667,3.900000,4.511111,4.211111,4.662963,4.618519
A59,4.727222,4.411111,4.900000,4.722222,4.900000,4.796296


In [295]:
dataset.to_excel('景区评分.xlsx')